In [1]:
import sys
import os
import numpy as np
from _3dpu_improved import *
ROOT = "../"

In [2]:
import yaml

# Load the YAML file
with open(ROOT + 'paths.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Access paths
data_path = config['paths']["data_big"]




In [3]:
import nibabel as nb
time = 3
data = nb.load(data_path).get_fdata()
data = np.array(data)[:60,:60,:60,time]

In [4]:
C = Resiuals(data)
C.map_nodes()
C.create_graph()
C.cycles = []
C.incycles = [1]*len(C.mapping)
C.connex =[1]*len(C.mapping)
C.detect_cycles()


100%|██████████| 160063/160063 [00:00<00:00, 352517.80it/s]


In [157]:
def dfs(C, v, path,path_dic,stack):
    C.visited[v] = True
    stack[v] = True
    path.append(v)
    path_dic[v] = len(path)-1

    for neighbour in C.Res_graph[v]:
        if not C.visited[neighbour]:
            if dfs(C,neighbour, path[:],path_dic,stack):
                return True
        elif stack[neighbour]:
            cycle_start_index = path_dic[neighbour]
            cycle = path[cycle_start_index:] + [neighbour]
            C.cycles.append(cycle)
            return True

    stack[v] = False
    return False

def detect_cycles(C,stack):
    C.visited = [False] * len(C.mapping)

    for i in tqdm(range(len(C.mapping))):
        stack = [False]*len(C.mapping)
        if not C.visited[i]:
            dfs(C,i, [],{},stack)
    

In [140]:
stack = [False]*len(C.mapping)

In [158]:
C.cycles = []
stack = [False]*len(C.mapping)
detect_cycles(C,stack)

 10%|█         | 16754/160063 [00:07<01:07, 2121.33it/s]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/mehdii/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_64870/4251330666.py", line 3, in <module>
    detect_cycles(C,stack)
  File "/tmp/ipykernel_64870/3243870059.py", line -1, in detect_cycles
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/mehdii/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mehdii/.local/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mehdii/.local/lib/python3.11/site-packages/IPython/core/ultratb.py

In [148]:
l,dic = ([1, 105868, 49312, 49331, 105909, 105910, 105911, 105912, 34, 50224, 1182, 51457, 107847, 1161, 50204, 106684, 16, 105888, 49313, 105869], {1: 1, 105867: 2, 49295: 3, 105868: 2, 2: 3, 1122: 4, 51389: 5, 49312: 3, 49331: 4, 105909: 5, 105910: 6, 105911: 7, 105912: 8, 34: 9, 50224: 10, 1182: 11, 51457: 12, 107847: 13, 1161: 14, 50204: 15, 106684: 16, 16: 17, 105888: 18, 49313: 19, 105869: 20})
dic

{1: 1,
 105867: 2,
 49295: 3,
 105868: 2,
 2: 3,
 1122: 4,
 51389: 5,
 49312: 3,
 49331: 4,
 105909: 5,
 105910: 6,
 105911: 7,
 105912: 8,
 34: 9,
 50224: 10,
 1182: 11,
 51457: 12,
 107847: 13,
 1161: 14,
 50204: 15,
 106684: 16,
 16: 17,
 105888: 18,
 49313: 19,
 105869: 20}

In [144]:
C.Res_graph[120]

[49401, 49421]

In [131]:

for cycle in C.cycles:
    for k in K[0]:
        if k in cycle:
            print(k,cycle)

In [97]:
stack[106684]

True

In [90]:
C.Res_graph[1182]

[51457]

In [6]:
Dict_of_rec = {}
for point in Points:
    Dict_of_rec[point] = []
for point in tqdm(Points):
    for i in range(len(C.cycles)):
        if point in C.cycles[i]:
            Dict_of_rec[point].append(i)


100%|██████████| 14824/14824 [00:15<00:00, 987.21it/s]


In [27]:
S = list(Points)
np.random.shuffle(S)
for point in S:
    assert len(Dict_of_rec[point]) == 1, point
        

AssertionError: 3797

In [28]:
for cycle in C.cycles:
    if 3797 in cycle:
        print((cycle))

[7134, 3797, 5995, 7134]
[112765, 7134, 3797, 5995, 112765]


In [38]:
C.cycles[6]

[89342, 35675, 35676, 89342]

In [39]:

C.Res_graph[89342]

[143810]

In [45]:
C.visited = [1]*len(C.mapping)
C.incycles = [1]*len(C.mapping)
C.dfs_iterative(613037)

In [48]:
C.incycles[613037]

1

In [6]:

def fill_open_paths(C):
    Not_in_cycles = []
    for node in range(len(C.mapping)):
        if C.incycles[node] != -1:
            Not_in_cycles.append(node)

    for node in tqdm(Not_in_cycles):  
        if C.incycles[node] == -1:
            continue

        path = []
        while C.incycles[node] != -1 and len(C.Res_graph[node]) > 0:
            path.append(node)
            node = C.Res_graph[node][0]  # Get the first/next node in the path

        # This condition ensures we don't add an empty path
        if len(path) > 1:
            C.open_paths.append(path)
            for point in path:
                C.incycles[point] = -1

In [7]:
fill_open_paths(C)

100%|██████████| 669794/669794 [00:01<00:00, 368730.64it/s]


In [8]:
Not_in_cycles = []
for node in range(len(C.mapping)):
    if C.incycles[node] != -1:
        Not_in_cycles.append(node)
(Not_in_cycles)[3]

604

In [65]:
C.cycles = []
C.incycles = [1]*len(C.mapping)
C.visited = [1]*len(C.mapping)
C.dfs_iterative(613037)

In [79]:
cpt

58

In [62]:
for cycle in C.cycles:
    if 1334354 in cycle:
        print(len(cycle))

7416


In [50]:
for cycle in C.cycles:
    if 613037 in cycle:
        print(cycle)

[613018, 613037, 1334354, 634, 6888, 1345645, 1345646, 613063, 1334377, 1334378, 613064, 1334321, 613017, 1334340, 613018]
[613037, 1334354, 634, 6888, 1345645, 1345646, 613063, 1334377, 1334378, 613064, 1334321, 613017, 1334340, 613037]


In [13]:
C.visited

[]

In [16]:
len(np.where(np.array(C.incycles) == 1)[0])

606743

In [15]:
for open_path in C.open_paths:
    assert C.is_boundary(open_path[0]), "you hav error"

In [17]:
L = [len(C.Res_graph[node]) for node in C.starting_open_paths]
L

[1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,


In [24]:
incycles = [1]*len(C.mapping)
def detect_cycles(C):
    path_indices = {}
    for v in tqdm(range((len(C.mapping)))):
        if C.connex[v] != -1:
            dfs_iterative(C,v,path_indices)

def dfs_iterative(C, start,path_indices):
    stack = [(start, -1)]
    path_indices[start] = 0
    path = [start]
    while stack:
        node, parent = stack.pop()
        if C.connex[node] != -1:
            for neighbour in C.Res_graph[node]:
                if neighbour != parent:
                    if neighbour in path:
                        cycle_start_index = path.index(neighbour)
                        C.cycles.append(path[cycle_start_index:] + [neighbour])
                        for point in path[cycle_start_index:] + [neighbour]:
                            C.connex[point] = -1
                            C.incycles[point] = -1
                    else:
                        stack.append((neighbour, node))
                        path = path + [neighbour]
                        path_indices[neighbour] = len(path)
                    

            C.connex[node] = -1



In [25]:
C.cycles = []
for point in range(len(C.mapping)):
    C.connex[point] = 2

detect_cycles(C)

100%|██████████| 2023266/2023266 [00:31<00:00, 64708.27it/s]  


In [41]:
len(np.where(np.array(incycles) == 1)[0])/len(C.mapping)

0.3310459425503122

In [40]:
669794/len(C.mapping)

0.3310459425503122

In [38]:
L = [len(compo) for compo in C.connected_components.values()]
bad = L.index(1)
colour = list(C.connected_components.keys())[bad]
C.connected_components[colour]

[28]

In [39]:
C.Res_graph[28]

[1338522, 1338523]

In [ ]:
from tqdm import tqdm
for r in tqdm(C.list_res):
    C.Res_graph[R[r]] = []

  0%|          | 0/6069798 [00:00<?, ?it/s]

100%|██████████| 6069798/6069798 [00:09<00:00, 632648.60it/s] 


In [ ]:
R

[0]

In [ ]:
for res in tqdm(C.list_res):
    i,j,k,axe,value = res
    # if i not in [0,X-1] and j not in [0,Y-1] and k not in [0,Z-1]:
    C.nodes_of_not_boundary(res)

  0%|          | 0/2023266 [00:00<?, ?it/s]


KeyError: 0

In [ ]:
C.identify_connected_component()

In [ ]:
D = (C.connected_components).values()
Number = [len(k) for k in D]
R = sorted(Number)
R

[0]

In [ ]:
R

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
Number

[2,
 1781553,
 3,
 5,
 6,
 2,
 2,
 2,
 5,
 2,
 2,
 5,
 1,
 5,
 5,
 1,
 4,
 2,
 8,
 4,
 2,
 6,
 2,
 4,
 10,
 4,
 4,
 4,
 4,
 16,
 6,
 4,
 6,
 15,
 8,
 4,
 4,
 8,
 4,
 4,
 12,
 4,
 2,
 3,
 6,
 24,
 4,
 12,
 6,
 4,
 4,
 2,
 4,
 4,
 6,
 4,
 4,
 8,
 8,
 22,
 4,
 14,
 8,
 6,
 2,
 4,
 6,
 2,
 10,
 4,
 4,
 4,
 2,
 7,
 4,
 8,
 10,
 14,
 4,
 4,
 6,
 6,
 6,
 4,
 9,
 4,
 3,
 8,
 4,
 2,
 12,
 6,
 2,
 2,
 4,
 6,
 10,
 4,
 4,
 9,
 6,
 4,
 8,
 4,
 4,
 10,
 6,
 6,
 6,
 4,
 4,
 6,
 6,
 1,
 6,
 4,
 2,
 10,
 4,
 6,
 4,
 4,
 4,
 6,
 4,
 4,
 14,
 6,
 4,
 1,
 4,
 4,
 6,
 3,
 4,
 4,
 6,
 4,
 4,
 6,
 10,
 5,
 4,
 4,
 14,
 4,
 4,
 10,
 6,
 12,
 8,
 4,
 16,
 2,
 8,
 3,
 6,
 4,
 4,
 4,
 4,
 14,
 12,
 6,
 4,
 4,
 4,
 4,
 4,
 10,
 9,
 4,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 6,
 4,
 4,
 10,
 4,
 4,
 4,
 5,
 6,
 10,
 6,
 5,
 4,
 6,
 6,
 4,
 4,
 8,
 11,
 4,
 11,
 2,
 4,
 14,
 3,
 4,
 2,
 3,
 1,
 2,
 1,
 1,
 3,
 5,
 8,
 4,
 3,
 5,
 2,
 3,
 4,
 7,
 2,
 2,
 23,
 2,
 3,
 7,
 2,
 2,
 16,
 3,
 3,
 3,
 4,
 2,
 2,
 6,
 2,
 19,
 3

In [ ]:
0 in (set(list(C.connex.values())))

False

In [ ]:
R = []
for key in tqdm(C.Res_graph.keys()):
    if len(C.Res_graph[key]) not in [0,1,2,3]:
        R.append(key)

100%|██████████| 2023266/2023266 [00:00<00:00, 2402570.20it/s]
